##  Live Chat API calls for Movie Recommendation System
### Moive collection since January 09, 1995 till September 26, 2018 

In [1]:
# DataSource :  https://grouplens.org/datasets/movielens/latest/
# Details: http://files.grouplens.org/datasets/movielens/ml-latest-README.html
# It contains 27753444 ratings and 1108997 tag applications across 58098 movies. 
# These data were created by 283228 users between January 09, 1995 and September 26, 2018. 
# This dataset was generated on September 26, 2018. 
# movies.csv , ratings.csv, tags.csv, links.csv ,genome-scores.csv ,genome-tags.csv

# consider user who has provided mimimum 100 movies rating
# consider movie whic has more than 1000 ratings

In [2]:
# During deployment of this model in to heroko seeing problem due to huge size of modle and trained vector
# model size give approxmatly 200MB this much huge data not accepted in github during commt time also
# we are going to reduce this dataset size and number of year of movies

# collect data from 2013 to till 2018 - 10 yrs movies 
# instead of 20 years data we taking 10yrs data

In [3]:
# import libraries
import pandas as pd
import numpy as np
import pickle

In [4]:
#Load dataset
movie_reviews=pd.read_csv("move_review_final_dataset.csv",compression="gzip")
movie_reviews

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
0,32930,category 6: day of destruction,"action , drama",2013,https://www.themoviedb.org/movie/32128,3,0,0,1,0,0,1,0,0,0,1,3.166667
1,40697,babylon 5,sci-fi,2013,https://www.themoviedb.org/movie/nan,200,13,1,2,4,5,12,28,48,26,61,3.895000
2,95595,bela kiss: prologue,"horror , mystery , thriller",2013,https://www.themoviedb.org/movie/155288,8,0,2,0,1,1,2,0,1,0,1,2.687500
3,99007,warm bodies,"comedy , horror , romance",2013,https://www.themoviedb.org/movie/82654,2168,51,44,60,141,188,440,456,459,173,156,3.331181
4,99335,addicted,drama,2013,https://www.themoviedb.org/movie/235271,13,0,1,1,1,1,1,3,4,0,1,3.192308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11431,193866,tales from the hood 2,horror,2018,https://www.themoviedb.org/movie/470614,1,0,1,0,0,0,0,0,0,0,0,1.000000
11432,193872,room laundering,drama,2018,https://www.themoviedb.org/movie/532677,1,0,0,0,0,0,0,0,1,0,0,4.000000
11433,193880,her name was mumu,drama,2016,https://www.themoviedb.org/movie/422666,1,0,0,0,1,0,0,0,0,0,0,2.000000
11434,193882,flora,"adventure , drama , horror , sci-fi",2017,https://www.themoviedb.org/movie/454439,1,0,0,0,1,0,0,0,0,0,0,2.000000


In [5]:
# Pass movie name and get index - used be NPL model 
# We can give movie name in any order and get index
movie_name_index_df=pd.read_csv("movie_user_rating_index.csv",encoding ='utf-8',compression='gzip')

In [6]:
# Testing time just give first 5 moive names from the below table 
# As we generated testing pivottable with this limited data
movie_name_index_df

,index,title
0,0,10 cloverfield lane
1,1,12 years a slave
2,2,22 jump street
3,3,about time
4,4,american hustle
...,...,...
107,107,world war z
108,108,"world's end, the"
109,109,x-men: apocalypse
110,110,x-men: days of future past


In [33]:
# pivot table was used in cosine similarity time during traing the Nearest Nighbour model
# So we need to use the same model while testing
# For testing we are ing movie_user_rating_pivottable_test.csv 
# but during deployment time we use movie_user_rating_pivottable_test.csv
#movie_user_rating_df=pd.read_csv("movie_user_rating_pivottable.csv", encoding ='utf-8',compression='gzip')
movie_user_rating_df=pd.read_csv("movie_user_rating_pivottable_test.csv", encoding ='utf-8',compression='gzip')

#### Load models

In [8]:
# We trained this model with cosine simility for movie recommendation system
model_knn = pickle.load( open( "collaborativeFiltering_model.pkl", "rb" ) )

In [9]:
# We trained this model with Navie Bayes - Multinomial 
# get moive name index which is used in model_knn
model_classifier = pickle.load( open( "collaborativeFiltering_NLP_model.pkl", "rb" ) )


In [10]:
#To get correct movie name, if user enter movie name mis order used in model_classifier
vectorizer = pickle.load(open("collaborativeFiltering_NLP_vectorizer.pkl", 'rb')) 

#### way to use models

In [11]:
# pass movie name and get movie index which can be used in recommendation modelmodel_knn
def getMovieIndex(name):
    name=name.strip().lower()
    firstValue="10 cloverfield lane".lower()
    test_vector= vectorizer.transform([name])
    predicted = model_classifier.predict(test_vector)
    
    available =   name in firstValue
    # return -1 for unavailable movies
    if((predicted[0] == 0) & (~available)):
        return -1
    else:
        return predicted[0]
    

In [12]:
# pass movie index and numb of movies recommendations 
def getRecomendedMoviesByIndex(queryIndex,numOfMoives=5):
    distances, indices = model_knn.kneighbors(movie_user_rating_df.iloc[queryIndex,:].values
                                              .reshape(1,-1), n_neighbors = numOfMoives)
    return distances,indices

In [13]:
# pass movie index and number of movie recommendations 
def recommendedMovies(name,numOfMoives=5):
    movieList=list()
    try:
        query_index=getMovieIndex(name)
        if(query_index == -1):
             return movieList
        distances, indices =getRecomendedMoviesByIndex(query_index,numOfMoives)
        for i in range(0, len(distances.flatten())):
            print(movie_name_index_df.title[indices.flatten()[i]],distances.flatten()[i])
            movieList.append(movie_name_index_df.title[indices.flatten()[i]])
        return movieList
    except:
         return movieList


In [14]:
# pass movie name and get movie details
def movieDetailsWithTitle(title):
    title=title.strip().lower()
    details = movie_reviews[movie_reviews["title"] == title]
    if(details.empty):
         details = movie_reviews[movie_reviews["title"].str.contains(title)]
        
   
    return details

In [15]:
def listToDataFrame(mvList):
    columns=["movieId","title","genres","year","tmdLink","total_rating","0.5","1.0","1.5","2.0","2.5","3.0","3.5","4.0","4.5","5.0","avg_rating"]
    movieList=list()
    for i, val in enumerate(mvList): 
        for rows in movieDetailsWithTitle(val).itertuples(): 
            movieList.append([rows.movieId, rows.title, rows.genres, rows.year,rows.tmdLink, rows.total_rating,
                                 rows._7,rows._8, rows._9, rows._10,rows._11,
                                 rows._12,rows._13,rows._14,rows._15,rows._16,rows.avg_rating])
    movies_df=pd.DataFrame(movieList,columns=columns)
    movies_df.sort_values(by="year", axis=0, ascending=False, inplace=True, kind='quicksort', na_position='last')
    return movies_df.reset_index(drop=True)
            

In [16]:
# find number of best movies
def findBestMovie(numberOfMovies=5):
    most_popular_movie=movie_reviews["total_rating"].max()
    bestMovie=movie_reviews.query("total_rating == @most_popular_movie")
    name=bestMovie.iloc[0].title
    #name="10 You Things  About I "
    mvList=recommendedMovies(name,numberOfMovies)
    return listToDataFrame(mvList)

In [17]:
findBestMovie(2)

interstellar  5.917488721252084e-14
the martian  0.2578723038276277


,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
0,134130,the martian,"adventure , drama , sci-fi",2015,https://www.themoviedb.org/movie/286217,16160,102,105,80,283,437,1295,2224,4845,3161,3628,4.043812
1,109487,interstellar,"sci-fi , imax",2014,https://www.themoviedb.org/movie/157336,23081,196,213,173,510,682,1893,2705,5387,4265,7057,4.092869


In [18]:
# resturn datafram as json
json=findBestMovie(2).to_json(orient = "records")
json

interstellar  5.917488721252084e-14
the martian  0.2578723038276277


'[{"movieId":134130,"title":"the martian ","genres":"adventure , drama , sci-fi","year":2015,"tmdLink":"https:\\/\\/www.themoviedb.org\\/movie\\/286217","total_rating":16160,"0.5":102,"1.0":105,"1.5":80,"2.0":283,"2.5":437,"3.0":1295,"3.5":2224,"4.0":4845,"4.5":3161,"5.0":3628,"avg_rating":4.043812},{"movieId":109487,"title":"interstellar ","genres":"sci-fi , imax","year":2014,"tmdLink":"https:\\/\\/www.themoviedb.org\\/movie\\/157336","total_rating":23081,"0.5":196,"1.0":213,"1.5":173,"2.0":510,"2.5":682,"3.0":1893,"3.5":2705,"4.0":5387,"4.5":4265,"5.0":7057,"avg_rating":4.092869}]'

In [19]:
# Get number of best movies with respect to given year
def findBestMovieWithYear(year,numberOfMovies=5):
    most_popular_movies_year=movie_reviews[movie_reviews["year"]==year].sort_values(by="total_rating", axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
    return most_popular_movies_year.iloc[:numberOfMovies]

In [20]:
findBestMovieWithYear(2018,2)

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
1426,122912,avengers: infinity war - part i,"action , adventure , sci-fi",2018,https://www.themoviedb.org/movie/299536,2668,50,49,27,64,80,232,373,601,503,689,3.956709
10246,187593,deadpool 2,"action , comedy , sci-fi",2018,https://www.themoviedb.org/movie/383498,1633,33,18,12,41,63,160,300,455,256,295,3.838947


In [21]:
# Get number of best movies with respect to given genres
def findBestMovieWithGenres(genres,numberOfMovies=5):
    genres= genres.replace("and",",")
    most_popular_movies_genres=  movie_reviews.query("genres == @genres").sort_values(by="total_rating", axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
    return most_popular_movies_genres.iloc[:numberOfMovies]

In [22]:
findBestMovieWithGenres("action and adventure",2)

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
763,112897,the expendables 3,"action , adventure",2014,https://www.themoviedb.org/movie/138103,1101,50,56,54,126,134,257,157,145,44,78,2.940509
5198,160563,the legend of tarzan,"action , adventure",2016,https://www.themoviedb.org/movie/258489,693,24,19,30,59,81,166,151,89,30,44,3.079365


In [23]:
# Get number of best movies with respect to genres and year
def findBestMovieWithGenresWithYear(genres,year,numberOfMovies=5):
    genres= genres.replace("and",",")
    most_popular_movies_genres_year=movie_reviews.query("genres == @genres & year == @year").sort_values(by="total_rating", axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
    return most_popular_movies_genres_year.iloc[:numberOfMovies]

In [27]:
findBestMovieWithGenresWithYear("comedy",2013,3)

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
85,102686,"hangover part iii, the",comedy,2013,https://www.themoviedb.org/movie/109439,1861,81,91,100,183,267,388,275,276,77,123,2.959162
369,107348,anchorman 2: the legend continues,comedy,2013,https://www.themoviedb.org/movie/109443,1660,80,74,75,181,211,353,295,231,73,87,2.948795
82,102481,"internship, the",comedy,2013,https://www.themoviedb.org/movie/116741,1472,37,51,58,113,193,327,302,225,82,84,3.119565


In [30]:
movieDetailsWithTitle("anchorman")

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
369,107348,anchorman 2: the legend continues,comedy,2013,https://www.themoviedb.org/movie/109443,1660,80,74,75,181,211,353,295,231,73,87,2.948795


In [31]:
# find number of similar (cosine similarity) movies with respect to given movie
def findSimilarMovies(name,mnumberOfMovies=5):
    mvList=recommendedMovies(name,mnumberOfMovies+1)
    return listToDataFrame(mvList)

In [32]:
findSimilarMovies("avengers",5)

avengers: age of ultron  2.0095036745715333e-14
captain america: the winter soldier  0.24629159213515806
ant-man  0.29215538293923216
iron man 3  0.3043386173059738
captain america: civil war  0.3125672510488062
guardians of the galaxy  0.3223693817166019


,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
0,188301,ant-man and the wasp,"action , adventure , comedy , fantasy , sci-fi",2018,https://www.themoviedb.org/movie/363088,391,4,5,2,8,16,64,106,111,45,30,3.645780
1,122918,guardians of the galaxy 2,"action , adventure , sci-fi",2017,https://www.themoviedb.org/movie/283995,4942,67,47,63,132,196,554,811,1327,774,971,3.859875
2,122920,captain america: civil war,"action , sci-fi , thriller",2016,https://www.themoviedb.org/movie/271110,5547,103,79,71,194,309,710,1005,1379,783,914,3.723544
3,122892,avengers: age of ultron,"action , adventure , sci-fi",2015,https://www.themoviedb.org/movie/99861,7331,180,117,118,289,492,1133,1430,1754,794,1024,3.588187
4,122900,ant-man,"action , adventure , sci-fi",2015,https://www.themoviedb.org/movie/102899,5806,106,65,82,223,329,912,1281,1601,644,563,3.590424
5,110102,captain america: the winter soldier,"action , adventure , sci-fi , imax",2014,https://www.themoviedb.org/movie/100402,8279,159,144,105,294,405,1064,1450,2275,1113,1270,3.709023
6,112852,guardians of the galaxy,"action , adventure , sci-fi",2014,https://www.themoviedb.org/movie/118340,15574,224,185,156,405,518,1501,2261,4075,2703,3546,3.932580
7,102125,iron man 3,"action , sci-fi , thriller , imax",2013,https://www.themoviedb.org/movie/68721,8028,167,140,149,378,586,1328,1593,1854,827,1006,3.536995
